## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('C:/Users/davec/Desktop/Class/6. WeatherPy with Python APIs/Challenge/WeatherPy_challenge.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,khatanga,RU,2020-12-05 19:18:46,71.97,102.50,4.95,94,100,18.90,light snow
1,1,pisco,PE,2020-12-05 19:18:46,-13.70,-76.22,73.40,69,0,8.05,clear sky
2,2,bluff,NZ,2020-12-05 19:18:46,-46.60,168.33,52.00,81,41,1.99,scattered clouds
3,3,codrington,AU,2020-12-05 19:18:46,-38.27,141.97,50.79,76,100,17.38,light rain
4,4,hithadhoo,MV,2020-12-05 19:18:46,-0.60,73.08,82.81,77,12,19.73,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp_prompt = float(input('What is the minimum temperature you would like for your trip? '))
max_temp_prompt = float(input('What is the maximum temperature you would like for your trip? '))                    

What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 76


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp_prompt) & (city_data_df["Max Temp"] >= min_temp_prompt)]
                                       
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,pisco,PE,2020-12-05 19:18:46,-13.70,-76.22,73.40,69,0,8.05,clear sky
2,2,bluff,NZ,2020-12-05 19:18:46,-46.60,168.33,52.00,81,41,1.99,scattered clouds
3,3,codrington,AU,2020-12-05 19:18:46,-38.27,141.97,50.79,76,100,17.38,light rain
6,6,busselton,AU,2020-12-05 19:18:47,-33.65,115.33,51.01,87,2,6.60,clear sky
7,7,buenos aires,AR,2020-12-05 19:17:40,-34.61,-58.38,70.00,48,20,9.17,few clouds
...,...,...,...,...,...,...,...,...,...,...,...
673,673,vaxjo,SE,2020-12-05 19:19:39,56.88,14.81,42.80,100,100,5.82,overcast clouds
675,675,sola,VU,2020-12-05 19:19:39,-13.88,167.55,75.20,100,75,4.23,broken clouds
677,677,la asuncion,VE,2020-12-05 19:19:39,11.03,-63.86,73.40,100,80,16.11,broken clouds
684,684,tateyama,JP,2020-12-05 19:19:40,34.98,139.87,46.40,81,75,8.05,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                287
City                   287
Country                283
Date                   287
Lat                    287
Lng                    287
Max Temp               287
Humidity               287
Cloudiness             287
Wind Speed             287
Current Description    287
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                283
City                   283
Country                283
Date                   283
Lat                    283
Lng                    283
Max Temp               283
Humidity               283
Cloudiness             283
Wind Speed             283
Current Description    283
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,pisco,PE,73.40,clear sky,-13.70,-76.22,
2,bluff,NZ,52.00,scattered clouds,-46.60,168.33,
3,codrington,AU,50.79,light rain,-38.27,141.97,
6,busselton,AU,51.01,clear sky,-33.65,115.33,
7,buenos aires,AR,70.00,few clouds,-34.61,-58.38,
8,mataura,NZ,52.00,broken clouds,-46.19,168.86,
11,hokitika,NZ,53.82,overcast clouds,-42.72,170.97,
13,ushuaia,AR,53.60,light rain,-54.80,-68.30,
14,lashio,MM,52.72,clear sky,22.93,97.75,
23,kahului,US,75.99,few clouds,20.89,-156.47,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,pisco,PE,73.40,clear sky,-13.70,-76.22,Hotel San Isidro Oficial
2,bluff,NZ,52.00,scattered clouds,-46.60,168.33,Bluff Homestead - Guesthouse & Campervan Park
3,codrington,AU,50.79,light rain,-38.27,141.97,Codrington Gardens Bed And Breakfast
6,busselton,AU,51.01,clear sky,-33.65,115.33,Observatory Guest House
7,buenos aires,AR,70.00,few clouds,-34.61,-58.38,Gran Hotel Argentino
8,mataura,NZ,52.00,broken clouds,-46.19,168.86,Ellie's Villa
11,hokitika,NZ,53.82,overcast clouds,-42.72,170.97,Beachfront Hotel Hokitika
13,ushuaia,AR,53.60,light rain,-54.80,-68.30,Albatros Hotel
14,lashio,MM,52.72,clear sky,22.93,97.75,Hotel CF Lashio - Burmese Only
23,kahului,US,75.99,few clouds,20.89,-156.47,Maui Seaside Hotel


In [10]:
# 8a. Create the output File (CSV)
# 8b. Export the City_Data into a csv

clean_hotel_df.to_csv(path_or_buf='C:/Users/davec/Desktop/Class/6. WeatherPy with Python APIs/Challenge/weatherPy_vacation.csv', index_label='City_ID')

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
gmaps.configure(api_key=g_key)

info_box_template = '''
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
'''

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows() ]
fig = gmaps.figure(center = (30.0,31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))